![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/reader/SparkNLP_Excel_Reader_Demo.ipynb)

# Introducing Excel reader in SparkNLP
This notebook showcases the newly added  `sparknlp.read().xls()` method in Spark NLP that parses Excel content from both local files and both local and distributed file systems into a Spark DataFrame.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!cp drive/MyDrive/JSL/sparknlp/sparknlp.jar .
!cp drive/MyDrive/JSL/sparknlp/spark_nlp-5.5.1-py2.py3-none-any.whl .

In [3]:
%env PYSPARK=3.4.0

env: PYSPARK=3.4.0


In [4]:
!pip install PYSPARK

In [5]:
!pip install spark_nlp-5.5.1-py2.py3-none-any.whl

Processing ./spark_nlp-5.5.1-py2.py3-none-any.whl


In [6]:
# import sparknlp
# # let's start Spark with Spark NLP
# spark = sparknlp.start()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("SparkNLP") \
    .master("local[*]") \
    .config("spark.driver.memory", "12G") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.kryoserializer.buffer.max", "2000M") \
    .config("spark.driver.maxResultSize", "0") \
    .config("spark.jars", "./sparknlp.jar") \
    .getOrCreate()


print("Apache Spark version: {}".format(spark.version))

Apache Spark version: 3.5.3


## Setup and Initialization
Let's keep in mind a few things before we start 😊

Support for reading html files was introduced in Spark NLP 5.5.2. Please make sure you have upgraded to the latest Spark NLP release.

For local files example we will download an Excel file from Spark NLP Github repo:

In [13]:
!mkdir excel-files
!wget https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/feature/SPARKNLP-1102-Adding-support-to-read-Excel-files/src/test/resources/reader/xls/vodafone.xlsx -P excel-files

mkdir: cannot create directory ‘excel-files’: File exists
--2024-12-19 18:05:41--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/feature/SPARKNLP-1102-Adding-support-to-read-Excel-files/src/test/resources/reader/xls/vodafone.xlsx
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12541 (12K) [application/octet-stream]
Saving to: ‘excel-files/vodafone.xlsx’

vodafone.xlsx       100%[===================>]  12.25K  --.-KB/s    in 0s      

2024-12-19 18:05:41 (70.5 MB/s) - ‘excel-files/vodafone.xlsx’ saved [12541/12541]



## Parsing Excel sheets from Local Files
Use the `xls()` method to parse Excel content from local directories.

In [14]:
import sparknlp
xls_df = sparknlp.read().xls("./excel-files")

xls_df.show()

+--------------------+--------------------+--------------------+
|                path|             content|                 xls|
+--------------------+--------------------+--------------------+
|file:/content/exc...|[50 4B 03 04 14 0...|[{Title, Financia...|
+--------------------+--------------------+--------------------+



In [15]:
xls_df.printSchema()

root
 |-- path: string (nullable = true)
 |-- content: binary (nullable = true)
 |-- xls: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- elementType: string (nullable = true)
 |    |    |-- content: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)



## Configuration Parameters
- Font Size: You can customize the font size used to identify paragraphs that should be treated as titles. By default, the font size is set to 9.
- Cell Separator: You can also customize the separator for each cell in the sheet. By defult, the separator is tab `"\t"`

However, if your Excel files require a different configuration, you can adjust this parameter accordingly. The example below demonstrates how to modify and work with this setting:

In [19]:
params = {"titleFontSize": "9", "cellSeparator": ";"}
xls_df = sparknlp.read(params).xls("./excel-files")
xls_df.select("xls").show(truncate=False)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [17]:
xls_df.printSchema()

root
 |-- path: string (nullable = true)
 |-- content: binary (nullable = true)
 |-- xls: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- elementType: string (nullable = true)
 |    |    |-- content: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)



You can also use DFS file systems like:
- Databricks: `dbfs://`
- HDFS: `hdfs://`
- Microsoft Fabric OneLake: `abfss://`